# Lab 4 – CNNs and Transfer Learning

This notebook strictly follows the lab instructions.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds

## Convolution and Pooling
Why: to visualize feature extraction and spatial reduction.

In [ ]:

(img_train, _), _ = keras.datasets.cifar10.load_data()
img = tf.image.resize(img_train[0], (224,224))
img = tf.expand_dims(tf.cast(img, tf.float32), axis=0)

kernel = tf.constant([[-1,0,1],[-1,0,1],[-1,0,1]], dtype=tf.float32)
kernel = tf.reshape(kernel, (3,3,1,1))
kernel = tf.repeat(kernel, 3, axis=2)

conv = tf.nn.conv2d(img, kernel, 1, "SAME")
max_pool = tf.nn.max_pool2d(conv, 2, 2, "SAME")

plt.imshow(conv[0]/255); plt.title("Convolution"); plt.axis("off")
plt.show()


## CNN on Fashion-MNIST
Why: baseline image classifier.

In [ ]:

(x_train,y_train),(x_test,y_test)=keras.datasets.fashion_mnist.load_data()
x_train=x_train/255.0

model = keras.Sequential([
    keras.layers.Input(shape=(28,28)),
    keras.layers.Reshape((28,28,1)),
    keras.layers.Conv2D(32,3,activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(10,activation='softmax')
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=3,validation_split=0.1)


## Transfer Learning with Xception
Why: reuse ImageNet-trained generic features.

In [ ]:

(ds_train,ds_val,_),_=tfds.load(
    "tf_flowers",
    split=["train[:70%]","train[70%:85%]","train[85%:]"],
    as_supervised=True,
    with_info=True
)

def preprocess(img,label):
    img=tf.image.resize(img,(299,299))
    img=keras.applications.xception.preprocess_input(img)
    return img,label

train_ds=ds_train.map(preprocess).batch(32)
val_ds=ds_val.map(preprocess).batch(32)

base_model=keras.applications.Xception(weights="imagenet",include_top=False)
base_model.trainable=False

model=keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(5,activation='softmax')
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(train_ds,validation_data=val_ds,epochs=3)


## ResNet
Why: demonstrate residual connections.

In [ ]:
keras.applications.ResNet50(weights=None).summary()